In [1]:
import pandas as pd
import numpy as np
import numpy.random as npr

In [2]:
data = pd.read_csv('SCI.TXT', header=None,
        usecols=[0,1,2,4,5,7,8,15,16,17,18])
columns_names = {0:'authors', 1:'title', 2:'source', 4:'language',
           5:'doctype', 7:'related_records_count',
           8:'references_count', 15:'rc', 16:'author_keywords',
           17:'editor_keywords', 18:'abstract'}
data = data.rename_axis(columns_names, axis=1)#.iloc[npr.choice(len(data), int(1e3))]

In [3]:
data.index = np.arange(len(data))
data.head()

,authors,title,source,language,doctype,related_records_count,references_count,rc,author_keywords,editor_keywords,abstract
0,Marchand-J Pigeon-M Bager-D Talbot-C,Influence of Chloride Solution Concentration o...,"ACI MATERIALS JOURNAL 1999, Vol 96, Iss 4, pp ...",English,Article,1,21,"UNIV-LAVAL, QUEBEC-CITY, PQ, CANADA/AALBORG-PO...",Deicers; Ice Formation; Porosity,HARDENED CEMENT PASTE,To analyze the influence of the chloride solut...
1,Edvardsen-C,Water Permeability and Autogenous Healing of C...,"ACI MATERIALS JOURNAL 1999, Vol 96, Iss 4, pp ...",English,Article,13,17,"COWI-CONSULTING-ENGINEERS, LYNGBY, DENMARK/PLA...",Autogenous Healing; Concretes; Cracking (Fract...,NaN,The well-known practical phenomenon of autogen...
2,Pauly-TM Lingvall-P,Effects of Mechanical Forage Treatment and Sur...,ACTA AGRICULTURAE SCANDINAVICA SECTION A-ANIMA...,English,Article,20,23,"SWEDISH-UNIV-AGR-SCI, DEPT ANIM NUTR & MANAGEM...",Chopping; Clostridium spp.; Forage Wagon; Harv...,CLOSTRIDIUM-TYROBUTYRICUM; QUALITY,Three silage experiments were conducted in Upp...
3,Mononen-J Harri-M Sepponen-J Korhonen-H Rekila...,A Top Box and a Floor Box as Breeding Nest Box...,ACTA AGRICULTURAE SCANDINAVICA SECTION A-ANIMA...,English,Article,9,20,"UNIV-KUOPIO, INST APPL BIOTECHNOL, BOX 1627, F...",Fur Farming; Fox Farming; Housing; Reproductio...,BEHAVIOR; PLATFORMS; SUCCESS,The aim of this study was to provide field inf...
4,Hindhede-J Mogensen-L Sorensen-JT,Effect of Group Composition and Feeding System...,ACTA AGRICULTURAE SCANDINAVICA SECTION A-ANIMA...,English,Article,3,15,"DANISH-INST-AGR-SCI, RES CTR FOULUM, DEPT ANIM...",Aggressive and Abnormal Behavior; Digital Diso...,SPACE ALLOWANCE,An experiment was conducted in eight Danish co...


In [4]:
data = data.fillna('')
data = data.join(pd.DataFrame(
    data['author_keywords'].apply(lambda x: x.lower().replace('-', ' ').
    replace('; ',' ').replace('(',' ').replace(')', ' ').split(sep=' ')) + 
    data['editor_keywords'].apply(lambda x: x.lower().replace('-', ' ').
    replace('; ',' ').replace('(',' ').replace(')', ' ').split(sep=' '))). 
    rename_axis({0:'keywords'}, axis=1))
keyword = list()
for klist in data['keywords']:
    seen = set()
    newklist = list()
    for k in klist:
        if k not in seen and len(k) > 2 and k.isalpha():
            newklist.append(k)
            seen.add(k)
    keyword.append(newklist)
data['keywords'] = pd.Series(keyword)
data['authors'] = data['authors'].apply(lambda x: x.lower().split())
data['language'] = data['language'].apply(lambda x: x.lower())
data['doctype'] = data['doctype'].apply(lambda x: x.lower())
data['title'] = data['title'].apply(lambda x: x.lower())
data['abstract'] = data['abstract'].apply(lambda x: x.lower())
data['source'] = data['source'].apply(lambda x: x.lower().split(sep=','))
journal = list()
year = list()
for sentence in data['source']:    
    journal.append(sentence[0][:-5])
    year.append(sentence[0][-4:])
data = data.join(pd.DataFrame(journal, columns=['journal']))
data = data.join(pd.DataFrame(year, columns=['article_year']))
data['rc'] = data['rc'].apply(lambda x: x.lower().split(sep='/'))
univ = list()
for list_sentences in data['rc']:
    u = list()
    for sentence in list_sentences:
        words = sentence.split(', ')
        u.append((words[0], words[-1]))
    univ.append(np.array(u))
data = data.join(pd.DataFrame(pd.Series(univ),columns=['university_country']))
data = data.drop(['author_keywords',
    'editor_keywords', 'rc', 'source'], axis=1)

In [5]:
data.shape

(364221, 11)

In [6]:
data.head()

,authors,title,language,doctype,related_records_count,references_count,abstract,keywords,journal,article_year,university_country
0,"[marchand-j, pigeon-m, bager-d, talbot-c]",influence of chloride solution concentration o...,english,article,1,21,to analyze the influence of the chloride solut...,"[deicers, ice, formation, porosity, hardened, ...",aci materials journal,1999,"[[univ-laval, canada], [aalborg-portland-inc, ..."
1,[edvardsen-c],water permeability and autogenous healing of c...,english,article,13,17,the well-known practical phenomenon of autogen...,"[autogenous, healing, concretes, cracking, fra...",aci materials journal,1999,"[[cowi-consulting-engineers, denmark], [planne..."
2,"[pauly-tm, lingvall-p]",effects of mechanical forage treatment and sur...,english,article,20,23,three silage experiments were conducted in upp...,"[chopping, clostridium, forage, wagon, harvest...",acta agriculturae scandinavica section a-anima...,1999,"[[swedish-univ-agr-sci, sweden]]"
3,"[mononen-j, harri-m, sepponen-j, korhonen-h, r...",a top box and a floor box as breeding nest box...,english,article,9,20,the aim of this study was to provide field inf...,"[fur, farming, fox, housing, reproduction, top...",acta agriculturae scandinavica section a-anima...,1999,"[[univ-kuopio, finland], [agr-res-ctr-finland,..."
4,"[hindhede-j, mogensen-l, sorensen-jt]",effect of group composition and feeding system...,english,article,3,15,an experiment was conducted in eight danish co...,"[aggressive, and, abnormal, behavior, digital,...",acta agriculturae scandinavica section a-anima...,1999,"[[danish-inst-agr-sci, denmark], [danish-inst-..."


In [7]:
language_series = pd.Series(sorted(set(data['language'])))
language = (pd.DataFrame(language_series.index).
    rename(columns={0:'language_id'}).join(pd.DataFrame(language_series).
    rename(columns={0:'language_name'})))
language.to_csv('language.csv', sep=';', index=False)

In [8]:
language.head()

,language_id,language_name
0,0,chinese
1,1,czech
2,2,danish
3,3,english
4,4,finnish


In [9]:
language.shape

(16, 2)

In [10]:
doctype_series = pd.Series(sorted(set(data['doctype'])))
doctype = (pd.DataFrame(doctype_series.index).rename(columns={0:'doctype_id'}).
    join(pd.DataFrame(doctype_series).rename(columns={0:'doctype_name'})))
doctype.to_csv('doctype.csv', sep=';', index=False)

In [11]:
doctype.head()

,doctype_id,doctype_name
0,0,article
1,1,bibliography
2,2,biographical-item
3,3,book-review
4,4,correction


In [12]:
doctype.shape

(14, 2)

In [13]:
journal_series = pd.Series(sorted(set(data['journal'])))
journal = (pd.DataFrame(journal_series.index).rename(columns={0:'journal_id'}).
    join(pd.DataFrame(journal_series).rename(columns={0:'journal_name'})))
journal.to_csv('journal.csv', sep=';', index=False)

In [14]:
journal[150:155]

,journal_id,journal_name
150,150,alcoholism-clinical and experimental research
151,151,algorithmica
152,152,alimentary pharmacology & therapeutics
153,153,allergy
154,154,alzheimer disease & associated disorders


In [15]:
journal.shape

(4386, 2)

In [16]:
author = list() 
for sentence in data['authors']:
    author.extend(sentence)
author_series = pd.Series(sorted(set([s.lower() for s in author])))
author = (pd.DataFrame(author_series.index).rename(columns={0:'author_id'}).
    join(pd.DataFrame(author_series).rename(columns={0:'author_name'})))
author.to_csv('author.csv', sep=';', index=False)

In [17]:
author[10000:10005]

,author_id,author_name
10000,10000,andersson-le
10001,10001,andersson-lf
10002,10002,andersson-lg
10003,10003,andersson-lhu
10004,10004,andersson-li


In [18]:
author.shape

(398972, 2)

In [19]:
author_dict = {v:k for k,v in author_series.iteritems()}
author_article_link = list()
for n, authors_of_article in enumerate(data['authors']):
    for aut in authors_of_article:
        author_article_link.append((author_dict[aut], n))
author_article_link = (pd.DataFrame(sorted(author_article_link)).
    rename(columns={0:'author_id', 1:'article_id'}))
author_article_link = author_article_link.drop_duplicates()
author_article_link.to_csv('map_article_author.csv', index=False, sep=';')

In [20]:
author_article_link.head()

,author_id,article_id
0,0,223842
1,1,305016
2,2,29950
3,3,261851
4,4,134681


In [21]:
author_article_link.shape

(2120156, 2)

In [22]:
author_pairs = pd.merge(author_article_link, author_article_link, on='article_id').query('author_id_x != author_id_y')
author_pairs.shape

(81889790, 3)

In [23]:
keyword = list() 
for sentence in data['keywords']:
    keyword.extend(sentence)
keyword_series = pd.Series(sorted(set(keyword)))
keyword = (pd.DataFrame(keyword_series.index).rename(columns={0:'keyword_id'}).
    join(pd.DataFrame(keyword_series).rename(columns={0:'keyword_name'})))
keyword.to_csv('keyword.csv', sep=';', index=False)

In [24]:
keyword.head()

,keyword_id,keyword_name
0,0,aaa
1,1,aaas
2,2,aab
3,3,aac
4,4,aadc


In [25]:
keyword.shape

(100671, 2)

In [26]:
keyword_dict = {v:k for k,v in keyword_series.iteritems()}
keyword_article_link = list()
for n, keywords_of_article in enumerate(data['keywords']):
    for kw in keywords_of_article:
        keyword_article_link.append((keyword_dict[kw], n))
keyword_article_link = (pd.DataFrame(sorted(keyword_article_link)).
    rename(columns={0:'keyword_id', 1:'article_id'}))
keyword_article_link = keyword_article_link.drop_duplicates()
keyword_article_link.to_csv('map_article_keyword.csv', index=False, sep=';')

In [27]:
keyword_article_link.head()

,keyword_id,article_id
0,0,24047
1,0,85073
2,0,86386
3,0,86916
4,0,86919


In [28]:
keyword_article_link.shape

(4827932, 2)

In [29]:
university, country = list(), list()
for pairs in data['university_country']:
    country.extend((pairs)[:,1])
    university.extend((pairs)[:,0])
university_series = pd.Series(sorted(set(university)))
university = (pd.DataFrame(university_series.index).
    rename(columns={0:'university_id'}).join(pd.DataFrame(university_series).
    rename(columns={0:'university_name'})))
university.to_csv('university.csv', header=True, sep='\t', index=False)
country_series = pd.Series(sorted(set(country)))
country = (pd.DataFrame(country_series.index).rename(columns={0:'country_id'}).
    join(pd.DataFrame(country_series).rename(columns={0:'country_name'})))
country.to_csv('country.csv', sep=';', index=False)

In [30]:
university[10000:10005]

,university_id,university_name
10000,10000,clin-hosp-gailefers
10001,10001,clin-hosp-lung-dis
10002,10002,clin-hosp-prof
10003,10003,clin-hosp-profesor-&-clin-servet
10004,10004,clin-hosp-rebro


In [31]:
university.shape

(69093, 2)

In [32]:
country.head()

,country_id,country_name
0,0,afghanistan
1,1,albania
2,2,algeria
3,3,andorra
4,4,angola


In [33]:
country.shape

(195, 2)

In [34]:
university_dict = {v:k for k,v in university_series.iteritems()}
country_dict = {v:k for k,v in country_series.iteritems()}
a_u_c_link = list()
for n, uc_pairs in enumerate(data['university_country']):
    for u, c in uc_pairs:
        a_u_c_link.append(np.array([n, university_dict[u], country_dict[c]]))
a_u_c_link = (pd.DataFrame(a_u_c_link).
    rename(columns={0:'article_id', 1:'university_id', 2:'country_id'}))
a_u_c_link = a_u_c_link.drop_duplicates()
a_u_c_link.to_csv('map_article_university_country.csv', index=False, sep=';')

In [35]:
a_u_c_link.shape

(1010037, 3)

In [36]:
a_u_c_link.head()

,article_id,university_id,country_id
0,0,64195,31
1,0,170,44
2,0,18845,182
3,1,11311,44
4,1,49255,44


In [37]:
language_dict = {v:k for k,v in language_series.iteritems()}
data['language'] = data['language'].apply(lambda x: language_dict[x])
journal_dict = {v:k for k,v in journal_series.iteritems()}
data['journal'] = data['journal'].apply(lambda x: journal_dict[x])
doctype_dict = {v:k for k,v in doctype_series.iteritems()}
data['doctype'] = data['doctype'].apply(lambda x: doctype_dict[x])
data = data.rename(columns={'language':'language_id',
                    'journal':'journal_id', 'doctype':'doctype_id'})
article = (pd.DataFrame(data.index).rename(columns={0:'article_id'}).
    join(data.drop(['authors', 'keywords',
    'university_country', 'keywords'], axis=1)))
article.to_csv('article.csv', sep=';', index=False)

In [38]:
article.head()

,article_id,title,language_id,doctype_id,related_records_count,references_count,abstract,journal_id,article_year
0,0,influence of chloride solution concentration o...,3,0,1,21,to analyze the influence of the chloride solut...,8,1999
1,1,water permeability and autogenous healing of c...,3,0,13,17,the well-known practical phenomenon of autogen...,8,1999
2,2,effects of mechanical forage treatment and sur...,3,0,20,23,three silage experiments were conducted in upp...,22,1999
3,3,a top box and a floor box as breeding nest box...,3,0,9,20,the aim of this study was to provide field inf...,22,1999
4,4,effect of group composition and feeding system...,3,0,3,15,an experiment was conducted in eight danish co...,22,1999


In [39]:
article.shape

(364221, 9)